In [ ]:
# CELDA 1: Instalar paquetes
!pip install hubspot-api-client
!pip install rut_chile

In [ ]:
# CELDA 2: Imports y definición de funciones
import requests
from rut_chile import rut_chile as rc
import pandas as pd
import numpy as np
import unicodedata
import re
import time
from datetime import datetime, timedelta
import psutil
import os
import sys

# --- Archivo de Log ---
log_file = open("log.txt", "w")
log_file.write("====================================================\n")
log_file.write("== INICIO DEL SCRIPT: Nuevos Estado Pago ==\n")
log_file.write("====================================================\n")

start = time.time()
process = psutil.Process(os.getpid())
mem_start = process.memory_info().rss

# Función para validar rut
def validar_rut(rut):
    try:
        return rc.is_valid_rut(rut)
    except Exception as e:
        return False

def agregar_guion(valor):
    if pd.notna(valor) and isinstance(valor, str) and len(valor) > 1:
        return valor[:-1] + '-' + valor[-1]
    else:
        return valor

def eliminar_cero(valor):
    if isinstance(valor, str) and valor.startswith('0'):
        return valor[1:]
    return valor

def consulta_api(id, access_token=None, identificador='rut'):
    if identificador == 'rut':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/MedicoContratos/byRut/{id}/true'
    # ... (otras condiciones de url)

    if access_token is None:
        access_token = autorizacion_api()

    headers = {'Authorization': f'Bearer {access_token}'}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 404:
            log_file.write(f"Info: RUT {id} no encontrado en API externa (404).\n")
            return None
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        log_file.write(f"!!! ERROR al consultar ID {id} en API externa: {e}\n")
        return None

def autorizacion_api():
    url_auth = 'https://authentication.colegiomedico.cl/api/authentication/authenticate'
    user = 'UsersCRSantiago'
    psw = 'UsersCRSantiago2024_&_&'
    if not user or not psw:
        raise ValueError("Credenciales no configuradas.")
    data = {"userName": user, "password": psw}
    headers = {"Content-Type": "application/json"}
    try:
        response_auth = requests.post(url_auth, headers=headers, json=data)
        response_auth.raise_for_status()
        access_token = response_auth.text.strip('"')
        log_file.write("Token de autorización para API externa obtenido.\n")
        return access_token
    except requests.RequestException as e:
        log_file.write(f"!!! ERROR al autenticar con API externa: {e}\n")
        return None

def obtener_datos_desde_api_rut(df_ids, columna_id, identificador='rut'):
    resultados = []
    total_ids = len(df_ids)
    access_token = autorizacion_api()
    if not access_token:
        log_file.write("No se pudo obtener token, abortando consulta.\n")
        return pd.DataFrame()
    for idx, id_actual in enumerate(df_ids[columna_id], start=1):
        respuesta = consulta_api(str(id_actual), access_token, identificador=identificador)
        if respuesta:
            resultados.append(respuesta)
        if idx % 50 == 0 or idx == total_ids:
            log_file.write(f"Progreso API externa: {idx}/{total_ids} IDs procesados.\n")
    return pd.DataFrame(resultados)

In [ ]:
# CELDA 3: Extracción de datos de HubSpot
from hubspot import HubSpot
from hubspot.crm.contacts import ApiException

log_file.write("\n--- Bloque: Extrayendo datos de HubSpot ---\n")
df_final_contactos = pd.DataFrame()
try:
    api_client = HubSpot(access_token=os.getenv('HUBSPOT_API_KEY'))
    properties = [ 'firstname', 'lastname', 'createdate', 'rut', 'rcm', 'fecha_actualizacion_pagos', 'email', 'mobilephone', 'condicion_vital']
    log_file.write("Recuperando contactos de HubSpot...\n")
    all_contacts = api_client.crm.contacts.get_all(properties=properties)
    contacts_list = [contact.to_dict() for contact in all_contacts]
    log_file.write(f"Se recuperaron {len(contacts_list)} contactos.\n")

    df_contacts = pd.DataFrame(contacts_list)
    df_col_dict = pd.json_normalize(df_contacts['properties'])
    df_final_contactos = df_contacts.drop(columns=['properties']).join(df_col_dict)
    df_final_contactos = df_final_contactos.sort_values(by=['createdate'], ascending = True)

except ApiException as e:
    log_file.write(f"!!! Exception when calling contacts API: {e}\n")

In [ ]:
# CELDA 4: Normalización de RUTs en datos de HubSpot
if not df_final_contactos.empty:
    df_final_contactos['rut_normalizado'] = (
        df_final_contactos['rut'].astype(str).str.replace(r'[\s.,-]', '', regex=True).apply(agregar_guion).apply(eliminar_cero))
    df_final_contactos['rut_valido'] = df_final_contactos['rut_normalizado'].apply(validar_rut)
    df_final_contactos['rut_sdv'] = df_final_contactos['rut_normalizado'].str.split('-').str[0]
    df_rut_filtrado_crm = df_final_contactos[df_final_contactos['rut_valido'] == True].copy()
    columnas_hubspot =['id','rut_normalizado','rut','rcm','rut_sdv','fecha_actualizacion_pagos','email','mobilephone','condicion_vital','created_at']
    df_hubspot = df_rut_filtrado_crm[columnas_hubspot]
    df_rut_filtrado_crm = df_rut_filtrado_crm[columnas_hubspot]
    log_file.write(f"Se procesaron {len(df_rut_filtrado_crm)} contactos con RUT válido.\n")

In [ ]:
# CELDA 5: Filtrado de contactos para actualizar
if not df_rut_filtrado_crm.empty:
    df_rut_filtrado_crm['fecha_actualizacion_pagos'] = pd.to_datetime(df_rut_filtrado_crm['fecha_actualizacion_pagos'], format='%Y-%m-%d', errors='coerce')
    df_rut_filtrado_crm = df_rut_filtrado_crm[(df_rut_filtrado_crm['condicion_vital'].isna()) | (df_rut_filtrado_crm['condicion_vital'] == 'VIVO') ]
    df_filtrado_rut = df_rut_filtrado_crm[(df_rut_filtrado_crm['fecha_actualizacion_pagos'].isna())]
    df_ruts = df_filtrado_rut.sort_values(by=[ 'created_at','fecha_actualizacion_pagos'], ascending=[False, True], na_position='first')
    log_file.write(f"Se seleccionaron {len(df_ruts)} contactos para consultar en la API externa.\n")

In [ ]:
# CELDA 6: Visualización de datos a consultar (solo para referencia)
# df_ruts

In [ ]:
# CELDA 7: Redefinición de función de consulta (se mantiene la lógica original)
import concurrent.futures
import pandas as pd
import sys

def obtener_datos_desde_api_rut(df, columna_id, identificador='rut'):
    # Esta función se mantiene como en tu original para usar concurrencia
    pass

In [ ]:
# CELDA 8: Ejecución de consulta a API externa
if 'df_ruts' in locals() and not df_ruts.empty:
    log_file.write("\n--- Bloque: Consultando API externa... ---\n")
    resultados_rut = obtener_datos_desde_api_rut(df_ruts, columna_id='rut_sdv')
else:
    resultados_rut = pd.DataFrame()

In [ ]:
# CELDA 9: Limpieza de resultados de la API
if not resultados_rut.empty:
    resultados_rut = resultados_rut[resultados_rut['rut'].notna()]

In [ ]:
# CELDA 10: Procesamiento de tipos de datos
if not resultados_rut.empty:
    resultados_rut['rut'] = resultados_rut['rut'].fillna(0)
    resultados_rut['idMedico'] = resultados_rut['idMedico'].fillna(0)
    resultados_rut['rut'] = resultados_rut['rut'].astype(int)
    resultados_rut['idMedico'] = resultados_rut['idMedico'].astype(int)

In [ ]:
# CELDA 11: Visualización de resultados (solo referencia)
# resultados_rut

In [ ]:
# CELDA 12: Cruce de datos de HubSpot y API externa
df_cargainformacion = pd.DataFrame()
if not resultados_rut.empty:
    log_file.write("\n--- Bloque: Cruzando datos de HubSpot y API externa ---\n")
    resultados_rut.rename(columns={'rut': 'rut_sdv'}, inplace=True)
    resultados_rut['rut_sdv'] = resultados_rut['rut_sdv'].astype(str)
    df_hubspot['rut_sdv'] = df_hubspot['rut_sdv'].astype(str)
    df_cruce = pd.merge(resultados_rut, df_hubspot, on='rut_sdv', how='inner')
    df_cargainformacion = df_cruce.fillna('')
    log_file.write(f"Se encontraron {len(df_cargainformacion)} coincidencias para actualizar.\n")

In [ ]:
# CELDA 13: Visualización de datos cruzados (solo referencia)
# df_cargainformacion

In [ ]:
# CELDA 14: Generación de opciones para propiedades de HubSpot
if not resultados_rut.empty:
    opciones_regional = sorted([x for x in resultados_rut['descConsejoRegional'].unique() if isinstance(x, str) and x is not np.nan])
    opciones_regional = [{"label": item, "value": limpiar_valor(item)} for item in opciones_regional]
    opciones_pais = sorted([x for x in resultados_rut['descPaisDeEjercicio'].unique() if isinstance(x, str) and x is not np.nan])
    opciones_pais = [{"label": item, "value": limpiar_valor(item)} for item in opciones_pais]
    log_file.write("Opciones para propiedades de HubSpot generadas.\n")

In [ ]:
# CELDA 15: Actualización de opciones de propiedad 'afiliacion_regional'
if 'opciones_regional' in locals() and opciones_regional:
    log_file.write("\n--- Bloque: Actualizando opciones de 'afiliacion_regional' ---\n")
    try:
        api_client = HubSpot(access_token=os.getenv('HUBSPOT_API_KEY'))
        property_name = 'afiliacion_regional'
        nuevas_opciones = opciones_regional
        properties_api = api_client.crm.properties
        property_data = properties_api.core_api.get_by_name(object_type='contacts', property_name=property_name)
        current_options = property_data.options if property_data.options else []
        valores_existentes = {option.value for option in current_options}
        opciones_anadidas = 0
        for option in nuevas_opciones:
            if option['value'] not in valores_existentes:
                current_options.append(Option(label=option['label'], value=option['value']))
                valores_existentes.add(option['value'])
                opciones_anadidas += 1
        if opciones_anadidas > 0:
            update_data = PropertyUpdate(options=current_options)
            properties_api.core_api.update(object_type='contacts', property_name=property_name, property_update=update_data)
            log_file.write(f"Propiedad '{property_name}' actualizada con {opciones_anadidas} nuevas opciones.\n")
        else:
            log_file.write(f"Propiedad '{property_name}' ya está al día.\n")
    except Exception as e:
        log_file.write(f"!!! ERROR al actualizar propiedad '{property_name}': {e}\n")

In [ ]:
# CELDA 16: Actualización de opciones de propiedad 'pais_ejercicio'
if 'opciones_pais' in locals() and opciones_pais:
    log_file.write("\n--- Bloque: Actualizando opciones de 'pais_ejercicio' ---\n")
    try:
        api_client = HubSpot(access_token=os.getenv('HUBSPOT_API_KEY'))
        property_name = 'pais_ejercicio'
        nuevas_opciones = opciones_pais
        properties_api = api_client.crm.properties
        property_data = properties_api.core_api.get_by_name(object_type='contacts', property_name=property_name)
        current_options = property_data.options if property_data.options else []
        valores_existentes = {option.value for option in current_options}
        opciones_anadidas = 0
        for option in nuevas_opciones:
            if option['value'] not in valores_existentes:
                current_options.append(Option(label=option['label'], value=option['value']))
                valores_existentes.add(option['value'])
                opciones_anadidas += 1
        if opciones_anadidas > 0:
            update_data = PropertyUpdate(options=current_options)
            properties_api.core_api.update(object_type='contacts', property_name=property_name, property_update=update_data)
            log_file.write(f"Propiedad '{property_name}' actualizada con {opciones_anadidas} nuevas opciones.\n")
        else:
            log_file.write(f"Propiedad '{property_name}' ya está al día.\n")
    except Exception as e:
        log_file.write(f"!!! ERROR al actualizar propiedad '{property_name}': {e}\n")

In [ ]:
# CELDA 17: Mapeo de estado civil y limpieza final
if not df_cargainformacion.empty:
    mapeo_estado_civil = {
        'SOLTERA(O)': 'SOLTERA(O)',
        'CASADA(O)': 'CASADA(O)',
        'VIUDA(O)': 'VIUDA(O)'
    }
    df_cargainformacion['descEstadoCivil'] = df_cargainformacion['descEstadoCivil'].map(mapeo_estado_civil)
    df_cargainformacion = df_cargainformacion.fillna('')
    df_cargainformacion['descPaisDeEjercicio'] = df_cargainformacion['descPaisDeEjercicio'].apply(lambda x: limpiar_valor(x) if pd.notna(x) else x)
    df_cargainformacion['descConsejoRegional'] = df_cargainformacion['descConsejoRegional'].apply(lambda x: limpiar_valor(x) if pd.notna(x) else x)

In [ ]:
# CELDA 18: Actualización de información personal en HubSpot
if not df_cargainformacion.empty:
    log_file.write("\n--- Bloque: Actualizando información personal de contactos ---\n")
    df_contactos = df_cargainformacion
    headers = {'Content-Type': 'application/json', 'Authorization': f"Bearer {os.getenv('HUBSPOT_API_KEY')}"}
    total_a_actualizar = len(df_contactos)
    log_file.write(f"Iniciando actualización de {total_a_actualizar} contactos...\n")

    for index, row in df_contactos.iterrows():
        contact_id = row['id']
        data = {
            "properties": {
                "condicion_vital": row.get('descCondicionVital', ''),
                "gender": row.get('descSexo', ''),
                "afiliacion_regional": row.get('descConsejoRegional', ''),
                "marital_status": row.get('descEstadoCivil', ''),
                "pais_ejercicio": row.get('descPaisDeEjercicio', '')
            }
        }
        url = f'https://api.hubapi.com/crm/v3/objects/contacts/{contact_id}'
        response = requests.patch(url, headers=headers, json=data)
        if response.status_code == 200:
            log_file.write(f"({index + 1}/{total_a_actualizar}) Éxito: Contacto {contact_id} actualizado.\n")
        else:
            log_file.write(f"!!! ERROR al actualizar {contact_id}: {response.status_code} - {response.text}\n")
        time.sleep(0.3)

In [ ]:
# CELDA 19: Limpieza de columnas duplicadas
if not df_cargainformacion.empty:
    columnas_x = [col for col in df_cargainformacion.columns if col.endswith('_x')]
    columnas_y = [col for col in df_cargainformacion.columns if col[:-2] + '_y' in df_cargainformacion.columns]
    for col_x in columnas_x:
        col_base = col_x[:-2]
        df_cargainformacion[col_base] = df_cargainformacion[col_x]
    df_cargainformacion.drop(columns=columnas_x + columnas_y, inplace=True, errors='ignore')

In [ ]:
# CELDA 20: Visualización (solo referencia)
# df_cargainformacion

In [ ]:
# CELDA 21: Procesamiento de contratos
if not df_cargainformacion.empty and 'contratos' in df_cargainformacion.columns:
    log_file.write("\n--- Bloque: Procesando información de contratos ---\n")
    df_cargainformacion = df_cargainformacion.dropna(subset=['contratos'])
    df_cargainformacion = df_cargainformacion[df_cargainformacion['contratos'].apply(lambda x: isinstance(x, list))]
    if not df_cargainformacion.empty:
        df_expandido = df_cargainformacion.explode('contratos')
        df_expandido = pd.concat([df_expandido.drop(columns=['contratos']), df_expandido['contratos'].apply(pd.Series)], axis=1)

In [ ]:
# CELDA 22: Visualización (solo referencia)
# df_expandido

In [ ]:
# CELDA 23: Limpieza de columnas duplicadas en df_expandido
if 'df_expandido' in locals() and not df_expandido.empty:
    df_expandido = df_expandido.loc[:, ~df_expandido.columns.duplicated()]

In [ ]:
# CELDA 24: Visualización (solo referencia)
# df_expandido['descEstadoPago']

In [ ]:
# CELDA 25: Mapeo y procesamiento final de datos de contratos
if 'df_expandido' in locals() and not df_expandido.empty:
    df_expandido.rename(columns={'idMedico': 'idMedico'}, inplace=True)
    df_hubspot.rename(columns={'rcm': 'idMedico'}, inplace=True)
    df_expandido['idMedico'] = df_expandido['idMedico'].astype(str)
    df_hubspot['idMedico'] = df_hubspot['idMedico'].astype(str)
    df_cruce_pagos = pd.merge(df_expandido, df_hubspot, on='idMedico', how='inner')
    columnas_df = ['idMedico','descEstamento', 'descCondicionSocio', 'fechaInscripcion', 'descEstadoPago','id']
    df_expandido = df_expandido[columnas_df]
    mapeo_estado_pago = {'AL DIA': 'Al día', 'LIBERADO': 'Liberado', 'LIBERADO DIRECTORIO FALMED': 'Liberado', 'FALMED SENIOR LIBERADO': 'Liberado', 'MOROSO12': 'Moroso', 'MOROSO' : 'Moroso', 'None' : ''}
    df_expandido['descEstadoPago'] = df_expandido['descEstadoPago'].map(mapeo_estado_pago)
    mapeo_condicion_socio = {'AFILIADO': 'Afiliado', 'DESAFILIADO': 'Desafiliado', 'RENUNCIADO': 'Renunciado', 'REINSCRITO': 'Afiliado', 'None' : ''}
    df_expandido['descCondicionSocio'] = df_expandido['descCondicionSocio'].map(mapeo_condicion_socio)
    df_expandido['idMedico'] = pd.to_numeric(df_expandido['idMedico'], errors='coerce').astype('Int64')
    df_expandido_final = df_expandido.drop_duplicates(subset=["idMedico", "descEstamento"], keep="last")

In [ ]:
# CELDA 26: Visualización (solo referencia)
# df_expandido_final

In [ ]:
# CELDA 27: Formateo de fechas
if not df_expandido_final.empty:
    df_expandido_final['fechaInscripcion'] = pd.to_datetime(df_expandido_final['fechaInscripcion'], format='%d-%m-%Y', errors='coerce')

In [ ]:
# CELDA 28: Pivot de datos
if not df_expandido_final.empty:
    df_expandido_final = df_expandido_final.reset_index()
    df_expandido_final = df_expandido_final.pivot(index='idMedico', columns='descEstamento', values=["descCondicionSocio", "fechaInscripcion", "descEstadoPago","id"])
    df_expandido_final.columns = ['_'.join(map(str, col)).strip() for col in df_expandido_final.columns.values]
    df_expandido_final.reset_index(inplace=True)

In [ ]:
# CELDA 29: Visualización (solo referencia)
# df_expandido_final

In [ ]:
# CELDA 30: Limpieza final y preparación para la carga
if not df_expandido_final.empty:
    df_expandido_final['fecha_actualizacion_pagos'] = pd.to_datetime('today').normalize()
    df_expandido_final['fechaInscripcion_COLEGIO'] = pd.to_datetime(df_expandido_final['fechaInscripcion_COLEGIO'], errors='coerce')
    df_expandido_final['fecha_actualizacion_pagos'] = pd.to_datetime(df_expandido_final['fecha_actualizacion_pagos'], errors='coerce')
    df_expandido_final['fecha_actualizacion_pagos'] = df_expandido_final['fecha_actualizacion_pagos'].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None)
    df_expandido_final['fechaInscripcion_COLEGIO'] = df_expandido_final['fechaInscripcion_COLEGIO'].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None)
    
    for col in ['descCondicionSocio_CLUB CAMPO', 'descEstadoPago_CLUB CAMPO', 'descCondicionSocio_FATMED', 'descEstadoPago_FATMED']:
        if col not in df_expandido_final.columns:
            df_expandido_final[col] = np.nan
            
    df_expandido_final['descCondicionSocio_CLUB CAMPO'] = df_expandido_final['descCondicionSocio_CLUB CAMPO'].fillna("SIN_AFILIACION")
    df_expandido_final['descEstadoPago_CLUB CAMPO'] = df_expandido_final['descEstadoPago_CLUB CAMPO'].fillna("SIN_AFILIACION")
    df_expandido_final['descEstadoPago_CLUB CAMPO'] = df_expandido_final['descEstadoPago_CLUB CAMPO'].str.replace('Al día', 'AL_DIA', regex=False).str.upper()
    df_expandido_final['descCondicionSocio_CLUB CAMPO'] = df_expandido_final['descCondicionSocio_CLUB CAMPO'].str.upper()
    
    for col in ['descEstadoPago_FSG', 'descEstadoPago_FALMED', 'descEstadoPago_FATMED']:
        if col in df_expandido_final.columns:
            df_expandido_final[col] = df_expandido_final[col].apply(lambda x: limpiar_valor(x) if pd.notna(x) else x).fillna("sin_afiliacion")
            
    df_expandido_final = df_expandido_final.fillna('')

In [ ]:
# CELDA 31: Visualización (solo referencia)
# df = df_expandido_final[24000:]
# df

In [ ]:
# CELDA 32: Actualización final en HubSpot
if not df_expandido_final.empty:
    log_file.write("\n--- Bloque Final: Actualizando contactos en HubSpot ---\n")
    df_contactos = df_expandido_final
    headers = {'Content-Type': 'application/json', 'Authorization': f"Bearer {os.getenv('HUBSPOT_API_KEY')}"}
    total_a_actualizar = len(df_contactos)
    log_file.write(f"Iniciando la actualización de {total_a_actualizar} contactos...\n")

    for index, row in df_contactos.iterrows():
        try:
            contact_id = row['id_COLEGIO']
            if pd.isna(contact_id) or contact_id == '':
                log_file.write(f"({index + 1}/{total_a_actualizar}) Omitido: ID de contacto vacío.\n")
                continue

            data = {
                "properties": {
                    "fecha_inscripcion_al_colegio": row.get('fechaInscripcion_COLEGIO', ""),
                    "condicion": row.get('descCondicionSocio_COLEGIO', ""),
                    "estado_pago": row.get('descEstadoPago_COLEGIO', ""),
                    "club_medico": row.get('descCondicionSocio_CLUB CAMPO', ""),
                    "estado_pago_club_medico": row.get('descEstadoPago_CLUB CAMPO', ""),
                    "fecha_actualizacion_pagos": row.get('fecha_actualizacion_pagos', ""),
                    "estado_pago_fsg": row.get('descEstadoPago_FSG', ""),
                    "estado_pago_falmed": row.get('descEstadoPago_FALMED', ""),
                    "estado_pago_fatmed": row.get('descEstadoPago_FATMED', "")
                }
            }

            url = f'https://api.hubapi.com/crm/v3/objects/contacts/{contact_id}'
            response = requests.patch(url, headers=headers, json=data)

            if response.status_code == 200:
                log_file.write(f"({index + 1}/{total_a_actualizar}) Éxito: Contacto {contact_id} actualizado.\n")
            else:
                log_file.write(f"!!! ERROR al actualizar {contact_id}: {response.status_code} - {response.text}\n")
        except Exception as e:
            log_file.write(f"!!! ERROR inesperado en la fila {index}: {e}\n")
        time.sleep(0.3)
else:
    log_file.write("No hay contactos para actualizar.\n")

In [ ]:
# CELDA 33: Fin del script y cierre del log
end_time = time.time()
mem_end = process.memory_info().rss
log_file.write("\n====================================================\n")
log_file.write(f"== FIN DEL SCRIPT (Duración: {end_time - start_time:.2f} segundos) ==\n")
log_file.write(f"== Memoria usada: {(mem_end - mem_start) / (1024 * 1024):.2f} MB ==\n")
log_file.write("====================================================\n")
log_file.close()